## Especialização em Inteligência Artificial – IFMG
## Análise de Séries Temporais - Lista de exercícios 1
**Aluno**: Alexandre Fortes Santana  

**Professor**: Carlos Severiano

[Notebook: https://github.com/alexandrefortes/series-temporais/blob/main/Lista%20de%20Exercicios%2002.ipynb](https://github.com/alexandrefortes/series-temporais/blob/main/Lista%20de%20Exercicios%2001.ipynb)

1 A série ilustrada abaixo, feita pelo IBGE, demonstra o volume de vendas mensal no Brasil. Ela está disponível no arquivo vendas_brasil.csv, em anexo a esta atividade, e também pode ser obtida em diferentes formatos no endereço: https://www.ibge.gov.br/estatisticas/economicas/comercio/9227-pesquisa-mensal-de-comercio.html?=&t=series-historicas